In [44]:
from pathlib import Path
import os

import fitz
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import load_chain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

In [ ]:
# Specify OPEN API key

os.environ["OPENAI_API_KEY"] = ""


In [4]:
file_name_pdf = Path(os.getcwd()).joinpath('suncorp-insurance-home-contents-insurance-product-disclosure-statement.pdf')
txt_file_name = Path(os.getcwd()).joinpath(file_name_pdf.stem + '.txt')

In [ ]:
# Convert pdf to text in memory
with fitz.open(file_name_pdf) as pdfreader:
    text = ""
    for page in pdfreader:
        text += page.get_text() + '\n'

In [9]:
# Cache text file
with open(txt_file_name, "w") as f:
    f.write(text)

In [6]:
# Read text file
with open(txt_file_name) as f:
    home_contents = f.read()

In [22]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(home_contents)

embeddings = OpenAIEmbeddings()
faiss_cache = f'faiss-cache-{txt_file_name.stem}'

docsearch = FAISS.from_texts(texts, embeddings)

In [46]:
query = "What is the maximum cover for a rug under each level"



In [47]:
docs = docsearch.similarity_search(query)

chain = load_qa_chain(OpenAI(temperature=0), chain_type='stuff')

chain.run(input_documents=docs, question=query)


KeyError: 'source'